In [1]:
import requests
from bs4 import BeautifulSoup
from HTMLParser import HTMLParser
import re

#GetUrl 클래스 : 인기순으로 정렬된 화면에서 각 와인에 대한 설명 페이지로 들어가는 링크를 리스트로 반환

class GetUrl(object):
    def __init__(self, url):
        self.url = url
    
    def get_url(self):
        res = requests.get(self.url)
        soup = BeautifulSoup(res.text)
        links = []
        
        for dd in soup.find_all('dd', attrs = {'class' : 'EngName'}):
            links.append(str(dd.a['href']))
        
        return links
    
    def make_url(self):
        links = self.get_url()
        wine_url = []
        url_base = 'http://wine21.com/13_WineSearch/winesearch01_view.php?sh_searchtype=2&sh_winetype=Red&sh_orderby=viewcnt|DESC&SelUno={}&page=1&increment=0'
        
        for i in links:
            val = re.search(r'\d{5}', i).group()
            wine_url.append(url_base.format(val))
        
        return wine_url

url = 'http://wine21.com/13_WineSearch/winesearch01_list.php?sh_searchtype=2&sh_winetype=Red&sh_orderby=viewcnt|DESC'  
#인기순으로 정렬한 페이지의 url 주소

wine = GetUrl(url)
url_list = wine.make_url()


#GetText 클래스 : 각 와인에 대한 설명 페이지의 url 리스트를 원소로 갖는다 (GetUrl 클래스에서 생성), 각 와인 이름을 key로 하고 포도종류, 당도 등을 value로 갖는 dict를 만든다.

from collections import OrderedDict

class GetText(object):
    def __init__(self, ulist):
        self.ulist = ulist
    
    def wine_info(self):
        wine_dict = OrderedDict()
        
        for url in self.ulist :
            res = requests.get(url)
            soup = BeautifulSoup(res.text)
            li = soup.find('li', attrs = {'class' : 'WineEndName'})
            wine = li.get_text()
            wine_dict[wine] = OrderedDict()
            
            li2 = soup.find('li', attrs = {'class' : 'Winery'})  #한글 인코딩 문제
            producer = li2.a.get_text()
            wine_dict[wine]['producer'] = producer
            
            li3 = soup.find('li', attrs = {'class' : 'Varieties'})
            varieties = ''
            for var in li3.find_all('a') :
                varieties += var.get_text()
            wine_dict[wine]['varieties'] = varieties
            
            li4 = soup.find('li', attrs = {'class' : 'BestFor'})  #앞 뒤 공백(\n) 문제
            bestfor = li4.get_text()
            wine_dict[wine]['bestfor'] = bestfor
            
            li5 = soup.find('li', attrs = {'class' : 'ServingTemp'})  #특수문자 표기 문제(ºC)
            servingtemp = li5.get_text()
            wine_dict[wine]['servingtemp'] = servingtemp
            
            li6 = soup.find('li', attrs = {'class' : 'Sweetness'})
            px = li6.find_all('img')[1]['style']
            wine_dict[wine]['sweetness'] = re.search(r'\d+', px).group()
            
            li7 = soup.find('li', attrs = {'class' : 'Body'})
            px = li7.find_all('img')[1]['style']
            wine_dict[wine]['body'] = re.search(r'\d+', px).group()
            
            ul = soup.find('ul', attrs = {'class' : 'TastingnoteList'})  #한글 깨져서 나옴
            wine_dict[wine]['tastingnote']= ul.get_text()
            
        return wine_dict

wine2 = GetText(url_list)
print wine2.wine_info()


c:\python27\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file c:\python27\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html5lib")

  markup_type=markup_type))


AttributeError: 'NoneType' object has no attribute 'find_all'

In [35]:
# -*- coding: utf-8 -*-
from bs4 import BeautifulSoup
import requests
import re
from HTMLParser import HTMLParser

In [40]:
response = requests.get("http://wine21.com/13_WineSearch/winesearch01_list.php")
dom = BeautifulSoup(response.text, "html.parser")

wine_names = dom.select("div#WineSearchList dd.EngName")

In [41]:
wine_name = wine_names[0]

In [42]:
wine_name.text

u'Genio Espanol Silver'

In [44]:
result = []

for wine_EngName in wine_names:
     result.append(wine_EngName.text)

print result,        

[u'Genio Espanol Silver', u'Genio Espanol Gold', u'Vinas Del Rey', u'Forman, Cabernet Sauvignon', u'Paso a Paso Tempranillo', u'Finca Luzon', u'Tarima', u'Anciens Temps Cabernet Syrah', u'Cannonball Cabernet Sauvignon', u'Chateau Le Vieux Chen', u'Manoir Grignon Sauvignon Blanc', u'Chatains Marte ']


In [ ]:
# http://wine21.com/13_WineSearch/winesearch01_list.php?sh_searchtype=1&page=3&increment=0
for i in range(1,1476):
    url = "http://wine21.com/13_WineSearch/winesearch01_list.php?sh_searchtype=1&page={}&increment=0".format(i)


    response = requests.get(url)
    dom = BeautifulSoup(response.text, "html.parser")

    wine_names = dom.select("div#WineSearchList dd.EngName")

    for wine_EngName in wine_names:
        result.append(wine_EngName.text)
        
len(result)        

In [ ]:
for i in range(37198,61674):
    url = "http://wine21.com/13_WineSearch/winesearch01_view.php?sh_searchtype=1&sh_orderby=writedate|DESC&sh_MaxPage=100&SelUno={}&page=1&increment=0".format(i)


    response = requests.get(url)
    dom = BeautifulSoup(response.text, "html.parser")

    wine_names = dom.select("div#WineViewTitle ul.WineName")

    for wine_name in wine_names:
        result.append(wine_EngName.text)
        
len(result)     

In [77]:
from bs4 import BeautifulSoup
import requests
import re
from HTMLParser import HTMLParser

url = "http://wine21.com/13_WineSearch/winesearch01_view.php?sh_searchtype=1&sh_orderby=writedate|DESC&sh_MaxPage=100&SelUno=61555&page=1&increment=0"


response = requests.get(url)
dom = BeautifulSoup(response.text, "html.parser")

wine_names = dom.select("div#WineViewTitle ul.WineName")
wine_name = wine_names[0]
wine_name

<ul class="WineName">\n<li class="WineKorName">\xec\x95\x84\xeb\xa5\xb4\xec\x88\x98\xec\x95\x84\xea\xb0\x80 \xed\x8c\x90 \xeb\x8d\xb0 \xec\x98\xa4\xeb\xa1\x9c 2014</li>\n<li class="WineEndName">Fan D. Oro 2014</li>\n<li class="WineProduction"><a href="javascript:ProductionWineListIt('P40');">Spain</a> &gt; <a href="javascript:ProductionWineListIt('P4047');">Ribera del Duero</a></li>\n</ul>

In [81]:
wine_name.split(">")

TypeError: 'NoneType' object is not callable